In [ ]:
!pip install pandas mlxtend

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
train = pd.read_csv('./eval_shortcut.csv', encoding='utf-8')

In [ ]:
feature_names = ['st_sim', 'bert_sim',
       'bert_info_ratio', 'tf-idf_info_ratio', 'num_q_ne',
       'num_a_ne',  'ne_unique_diff', 'len_q_token', 'len_a_token']
x = train.loc[:, feature_names]
y = train.loc[:,'rating_shortcut']

In [ ]:
# 3
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

def discretize(df, bins=3):
    for col in df.columns:
        if df[col].dtype == 'float64' or df[col].dtype == 'int64':
            df[col] = pd.qcut(df[col], bins, duplicates='drop')
    return df


high_rating_threshold = y.quantile(0.7)
y_high = y >= high_rating_threshold


X_discretized = discretize(x.copy())
df = X_discretized.copy()
df['high_rating'] = y_high


df_dummies = pd.get_dummies(df)

frequent_itemsets = apriori(df_dummies, min_support=0.05, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

high_rating_rules = rules[rules['consequents'].apply(lambda x: 'high_rating' in str(x))]

high_rating_rules = high_rating_rules.sort_values(by='confidence', ascending=False)

for _, rule in high_rating_rules.head(3).iterrows():
    antecedents = rule['antecedents']
    consequents = rule['consequents']
    support = rule['support']
    confidence = rule['confidence']
    lift = rule['lift']
    print(f"Rule: {antecedents} => {consequents}")
    print(f"Support: {support:.2f}, Confidence: {confidence:.2f}, Lift: {lift:.2f}\n")

    
data = pd.read_csv('./22-8.csv', encoding='utf-8')
def filter_dataframe(df, conditions):
    condition_strs = []
    for cond in conditions:
        feature, bin_range = cond.split('_(')
        bin_range = bin_range.rstrip(']')
        lower, upper = map(float, bin_range.split(', '))
        condition_strs.append(f'({lower} <= `{feature}`) & (`{feature}` < {upper})')
    query_str = ' & '.join(condition_strs)
    return df.query(query_str)


filtered_dfs = []
for _, rule in high_rating_rules.head(3).iterrows():
    conditions = rule['antecedents']
    filtered_df = filter_dataframe(data, conditions)
    filtered_dfs.append(filtered_df)

for i, filtered_df in enumerate(filtered_dfs):
    if i > 0:
        break
    print(filtered_df.shape)
    filtered_df.to_csv(f'0525_report_filtered_df_3_.csv', index=False)


In [ ]:
data = pd.read_csv('./22-8.csv', encoding='utf-8')

In [ ]:
data = pd.read_csv('./22-8.csv', encoding='utf-8')

def filter_dataframe(df, conditions):
    condition_strs = []
    for cond in conditions:
        feature, bin_range = cond.split('_(')
        bin_range = bin_range.rstrip(']')
        lower, upper = map(float, bin_range.split(', '))
        condition_strs.append(f'({lower} <= `{feature}`) & (`{feature}` < {upper})')
    query_str = ' & '.join(condition_strs)

    return df.query(query_str)


filtered_dfs = []
for _, rule in high_rating_rules.head(3).iterrows():
    conditions = rule['antecedents']
    filtered_df = filter_dataframe(data, conditions)
    filtered_dfs.append(filtered_df)

for i, filtered_df in enumerate(filtered_dfs):
    if i > 0:
        break
    print(filtered_df.shape)
    filtered_df.to_csv(f'5025_report_filtered_df_1_.csv', index=False)


In [ ]:
data1 = pd.read_csv('./filtered_df_rule_1.csv', encoding='utf-8')
data1.shape